# Term Frequency
Term frequency computes the number of times that a word or term occurs in a document.  Term frequency is often used as part of a larger text processing pipeline, which may include operations such as stemming, stop word removal and topic modelling.

In [36]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [37]:
# Greenplum 4.3.10.0
# %sql postgresql://gpdbchina@10.194.10.68:61000/madlib
        
# PostgreSQL local
%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: fmcquillan@madlib'

In [38]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.13, git revision: unknown, cmake configuration time: Wed Dec 20 08:02:21 UTC 2017, build type: Release, build system: Darwin-17.3.0, C compiler: Clang, C++ compiler: Clang"


# 1.  Prepare documents
First we create a document table with one document per row:

In [58]:
%%sql
DROP TABLE IF EXISTS documents;
CREATE TABLE documents(docid INT4, contents TEXT);

INSERT INTO documents VALUES
(0, 'I like to eat broccoli and bananas. I ate a banana and spinach smoothie for breakfast.'),
(1, 'Chinchillas and kittens are cute.'),
(2, 'My sister adopted two kittens yesterday.'),
(3, 'Look at this cute hamster munching on a piece of broccoli.');

SELECT * from documents ORDER BY docid;

Done.
Done.
4 rows affected.
4 rows affected.


docid,contents
0,I like to eat broccoli and bananas. I ate a banana and spinach smoothie for breakfast.
1,Chinchillas and kittens are cute.
2,My sister adopted two kittens yesterday.
3,Look at this cute hamster munching on a piece of broccoli.


You can apply stemming, stop word removal and tokenization at this point in order to prepare the documents for text processing. Depending upon your database version, various tools are available here. Databases based on more recent versions of PostgreSQL may do something like:

In [53]:
%%sql
SELECT tsvector_to_array(to_tsvector('english',contents)) from documents;

4 rows affected.


tsvector_to_array
"[u'ate', u'banana', u'breakfast', u'broccoli', u'eat', u'like', u'smoothi', u'spinach']"
"[u'chinchilla', u'cute', u'kitten']"
"[u'adopt', u'kitten', u'sister', u'two', u'yesterday']"
"[u'broccoli', u'cute', u'hamster', u'look', u'munch', u'piec']"


In this example, we assume a database based on an older version of PostgreSQL and just perform basic punctuation removal and tokenization. The array of words is added as a new column to the documents table:

In [59]:
%%sql
ALTER TABLE documents ADD COLUMN words TEXT[];

UPDATE documents SET words = 
    regexp_split_to_array(lower(
    regexp_replace(contents, E'[,.;\']','', 'g')
    ), E'[\\s+]');
    
SELECT * FROM documents ORDER BY docid;

Done.
4 rows affected.
4 rows affected.


docid,contents,words
0,I like to eat broccoli and bananas. I ate a banana and spinach smoothie for breakfast.,"[u'i', u'like', u'to', u'eat', u'broccoli', u'and', u'bananas', u'i', u'ate', u'a', u'banana', u'and', u'spinach', u'smoothie', u'for', u'breakfast']"
1,Chinchillas and kittens are cute.,"[u'chinchillas', u'and', u'kittens', u'are', u'cute']"
2,My sister adopted two kittens yesterday.,"[u'my', u'sister', u'adopted', u'two', u'kittens', u'yesterday']"
3,Look at this cute hamster munching on a piece of broccoli.,"[u'look', u'at', u'this', u'cute', u'hamster', u'munching', u'on', u'a', u'piece', u'of', u'broccoli']"


# 2.  Term frequency
Build a histogram for each document:

In [60]:
%%sql
DROP TABLE IF EXISTS documents_tf, documents_tf_vocabulary;

SELECT madlib.term_frequency('documents',    -- input table
                             'docid',        -- document id
                             'words',        -- vector of words in document
                             'documents_tf'  -- output table
                            );

SELECT * FROM documents_tf ORDER BY docid;

Done.
1 rows affected.
36 rows affected.


docid,word,count
0,a,1
0,breakfast,1
0,banana,1
0,and,2
0,eat,1
0,smoothie,1
0,to,1
0,like,1
0,broccoli,1
0,bananas,1


# 3.  Term frequency with vocabulary

In this example we create a vocabulary of the words and store a wordid in the output table instead of the actual word.

In [61]:
%%sql
DROP TABLE IF EXISTS documents_tf, documents_tf_vocabulary;

SELECT madlib.term_frequency('documents',    -- input table
                             'docid',        -- document id
                             'words',        -- vector of words in document
                             'documents_tf',-- output table
                             TRUE
                            );

SELECT * FROM documents_tf ORDER BY docid;

Done.
1 rows affected.
36 rows affected.


docid,wordid,count
0,17,1
0,9,1
0,25,1
0,12,1
0,13,1
0,15,2
0,0,1
0,2,2
0,28,1
0,5,1


Note above that a wordid have been generated.  The vocabulary table maps wordid to the actual word text:

In [62]:
%%sql
SELECT * FROM documents_tf_vocabulary ORDER BY wordid;

31 rows affected.


wordid,word
0,a
1,adopted
2,and
3,are
4,at
5,ate
6,banana
7,bananas
8,breakfast
9,broccoli
